In [1]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import pickle
from sklearn import naive_bayes, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<h2>lấy dữ liệu đã được xử lý</h2>
<p>dữ liệu phục vụ cho mô hình là một csr matrix, hay nói cách khác là một tập hợp gồm n phần tử, mỗi phần tử là một vector p chiều</p>
<p>mỗi chiều của vector là một từ trong tiếng việt, bao gồm cả từ đơn và từ ghép</p>
<p>file .pkl dùng để lưu lại các object(list, tuple ...) trong python</p>
<p><strong>count_vect_data lưu lại một ma trận có 40000 cột và số hàng tương ứng với số văn bản, mỗi phần tử là số lần xuất hiện của từ trong văn bản</strong><p>
<p><strong>tfidf như cont_vect, nhưng giá trị phần tử là tần suất thay vì số lần xuất hiện</strong></p>
<p><strong>tfidf_svd là tfidf nhưng đã giảm số chiều(số cột)</strong></p>

In [2]:
X_data_count = pickle.load(open("count_vect_data.pkl", "rb"))
X_test_count = pickle.load(open("count_vect_test.pkl", "rb"))
X_data_tfidf_svd = pickle.load(open("X_data_tfidf_svd.pkl", "rb"))
X_test_tfidf_svd = pickle.load(open("X_test_tfidf_svd.pkl", "rb"))
y_data = pickle.load(open("raw_data/y_data.pkl", "rb"))
y_test = pickle.load(open("raw_data/y_test.pkl", "rb"))
X_data = pickle.load(open("raw_data/X_data.pkl", "rb"))
X_test = pickle.load(open("raw_data/X_test.pkl", "rb"))
X_data_tfidf = pickle.load(open("X_data_tfidf.pkl", "rb"))

<h2>đánh nhãn cho các lớp, do model hoạt động dựa trên dữ liệu số</h2>


In [3]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ # kết quả: array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
                 #                 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
                 #                 'Vi tinh'], dtype='<U16')

#cell nay bo cx dc, cac thuat toan van chay binh thuong

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

<h2>build hàm huấn luyện chung để cho các model để thuận tiện thay đổi model</h2>

In [4]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)

        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)

 
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

<h1>Tiến hành build các model</h1>

<h2>1, Naive-bayes</h2>

In [5]:
naivBayesType = naive_bayes.MultinomialNB()
train_model(naivBayesType, X_data_count, y_data, X_test_count, y_test, is_neuralnet=False)

Validation accuracy:  0.9164490861618799
Test accuracy:  0.7852889667250438


<h2>2, SVM</h2>
<p>svm và knn test dữ liệu có số chiều nhỏ<p>

In [6]:
from sklearn import svm
svmModel = svm.SVC()
train_model(svmModel, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Validation accuracy:  0.9295039164490861
Test accuracy:  0.7957968476357268


<h2>3, KNN</h2>


In [7]:
from sklearn import neighbors
NB_model = neighbors.KNeighborsClassifier()
train_model(NB_model, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Validation accuracy:  0.8459530026109661
Test accuracy:  0.6612959719789843


<h2> sử dụng model </h2>

<p>tạo mẫu</p>

các tư thế xử lý dữ liệu

In [8]:
from sklearn.decomposition import TruncatedSVD
#chuyển data về kiểu tfidf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
#chuyển data về kiểu giảm số chiều
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)
#chuyển data về kiểu đếm số lần xuất hiện 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)



CountVectorizer(token_pattern='\\w{1,}')

<p>preprocessing theo kiểu tiếng việt</p>

In [9]:
test = ["ManCity đang có phong độ tốt trước trận tứ kết lượt đi Champions League trên sân nhà gặp Bayern. Đoàn quân của Pep Guardiola hiểu rằng họ cần có một kết quả tốt trước khi làm khách trên sân đối thủ ở lượt về, vì vậy ManCity chủ động dâng cao đội hình tấn công gây sức ép lên đối phương ngay sau tiếng còi khai cuộc. Phút 5, De Bruyne chuyền bóng cho Haaland từ bên trái. Cầu thủ người Na Uy bứt tốc nhanh và dứt điểm uy lực từ bên trái vòng cấm địa, bóng đi vọt xà. 9 phút sau, Haaland khiến cổ động viên Bayern thót tim khi lao vào rất nhanh tranh bóng với thủ thành Sommer ngay trên vạch vôi. Thủ môn đội khách xử lý chậm, anh phá bóng đập chân Haaland nhưng may mắn trái bóng đã văng ra biên. Haaland thi đấu nổi bật trên hàng công của Man City, các đồng đội cũng tập trung nhiều bóng cho anh. Phút 22, Haaland thoải mái dứt điểm ở đầu vòng cấm địa. Tiếc rằng cú sút của anh hơi nhẹ, bóng đi chìm và hướng thẳng vào vị trí của Sommer nên không gây nguy hiểm."]
lines = gensim.utils.simple_preprocess(test[0])#loại bỏ các phần không liên quan, ex: dấu chấm, dấu phẩy
lines = " ".join(lines)
lines = ViTokenizer.tokenize(lines)
test[0] = lines
print(lines)

mancity đang có phong_độ tốt trước trận tứ_kết lượt đi champions league trên sân_nhà gặp bayern đoàn quân của pep guardiola hiểu rằng họ cần có một kết_quả tốt trước khi làm_khách trên sân đối_thủ lượt về vì_vậy mancity chủ_động dâng cao đội_hình tấn_công gây sức_ép lên đối_phương ngay sau tiếng còi khai_cuộc phút de bruyne chuyền bóng cho haaland từ bên trái cầu_thủ người na uy bứt tốc nhanh và dứt_điểm uy_lực từ bên trái vòng cấm_địa bóng đi vọt xà phút sau haaland khiến cổ_động_viên bayern thót_tim khi lao vào rất nhanh tranh bóng với thủ_thành sommer ngay trên vạch vôi thủ_môn đội khách xử_lý chậm anh phá bóng đập chân haaland nhưng may_mắn trái bóng đã văng ra biên haaland thi_đấu nổi_bật trên hàng công của man city các đồng_đội cũng tập_trung nhiều bóng cho anh phút haaland thoải_mái dứt_điểm đầu vòng cấm_địa tiếc rằng cú sút của anh hơi nhẹ bóng đi chìm và hướng thẳng vào vị_trí của sommer nên không gây nguy_hiểm


<p>chuyển kiểu dữ liệu tf-idf</p>

In [10]:

example_count_type = count_vect.transform(test)

example = tfidf_vect.transform(test)
print(example)
example_truncated = svd.transform(example)

  (0, 29708)	0.07615150639699281
  (0, 29706)	0.0544543962413243
  (0, 29647)	0.08356450116102593
  (0, 29510)	0.06817812610997999
  (0, 29506)	0.0765673596366847
  (0, 29254)	0.07865264364011168
  (0, 29177)	0.03966085555219432
  (0, 28715)	0.02041243906034603
  (0, 28500)	0.06412146901563318
  (0, 28314)	0.10226193451601755
  (0, 28269)	0.030117245117866197
  (0, 27633)	0.060782945455345246
  (0, 27326)	0.10476767163441152
  (0, 26670)	0.020423278931536757
  (0, 26645)	0.0959523158558891
  (0, 26637)	0.058819195491514174
  (0, 26614)	0.02489786812620885
  (0, 26494)	0.08984034652738486
  (0, 26388)	0.11534806111850102
  (0, 26304)	0.11534806111850102
  (0, 26217)	0.1041155280265055
  (0, 26196)	0.05938003604011965
  (0, 26147)	0.04854003608338226
  (0, 26128)	0.036121466692677164
  (0, 25530)	0.11534806111850102
  :	:
  (0, 6570)	0.10150190837305864
  (0, 6387)	0.06418981395394735
  (0, 6345)	0.03957840401279363
  (0, 6284)	0.1936749761034522
  (0, 6023)	0.024481345709388775
  (0, 59

In [11]:
print(len(example_truncated))

1


dự đoán

In [12]:
print("kết quả của naive-bayes model: {}".format(naivBayesType.predict(example_count_type)))
print("kết quả của SVM model: {}".format(svmModel.predict(example_truncated)))
print("kết quả của KNN model: {}".format(NB_model.predict(example_truncated)))

kết quả của naive-bayes model: ['The thao']
kết quả của SVM model: ['The thao']
kết quả của KNN model: ['The thao']


<h2>keyword extraction</h2>

In [13]:
feature_names = tfidf_vect.get_feature_names_out()
feature_scores = example.toarray()[0]
# Sort the features by their scores
sorted_indices = feature_scores.argsort()[::-1]
# Print the top 5 features and their scores
for i in range(5):
    print(feature_names[sorted_indices[i]], feature_scores[sorted_indices[i]])

bóng 0.40895724566074687
bayern 0.25206173354962336
trái 0.20103160734484218
cấm_địa 0.1936749761034522
dứt_điểm 0.1706720230636951


<h2> text mining</h2>
<strong> cần tối ưu - cần tiền xử lí văn bản </strong>
<p> hiện chưa thể làm việc với văn bản lớn </p>

In [14]:
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules
# import pandas as pd
# # Define the documents
# documents = X_data

# # Tokenize the documents
# tokenized_documents = [document.lower().split() for document in documents]

# # Encode the tokenized documents
# te = TransactionEncoder()
# te_ary = te.fit(tokenized_documents).transform(tokenized_documents)
# df = pd.DataFrame(te_ary, columns=te.columns_)

# # Find frequent itemsets using Apriori algorithm
# frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# # Generate association rules
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

# # Print the association rules
# #print(rules)

<h2>text summarization</h2>

<p>example:</p>

In [ ]:
import nltk
nltk.download('punkt')

In [38]:
text = ["Man City đang có phong độ tốt trước trận tứ kết lượt đi Champions League trên sân nhà gặp Bayern. Đoàn quân của Pep Guardiola hiểu rằng họ cần có một kết quả tốt trước khi làm khách trên sân đối thủ ở lượt về, vì vậy Man City chủ động dâng cao đội hình tấn công gây sức ép lên đối phương ngay sau tiếng còi khai cuộc.Phút 5, De Bruyne chuyền bóng cho Haaland từ bên trái. Cầu thủ người Na Uy bứt tốc nhanh và dứt điểm uy lực từ bên trái vòng cấm địa, bóng đi vọt xà. 9 phút sau, Haaland khiến cổ động viên Bayern thót tim khi lao vào rất nhanh tranh bóng với thủ thành Sommer ngay trên vạch vôi. Thủ môn đội khách xử lý chậm, anh phá bóng đập chân Haaland nhưng may mắn trái bóng đã văng ra biên. Haaland thi đấu nổi bật trên hàng công của Man City, các đồng đội cũng tập trung nhiều bóng cho anh. Phút 22, Haaland thoải mái dứt điểm ở đầu vòng cấm địa. Tiếc rằng cú sút của anh hơi nhẹ, bóng đi chìm và hướng thẳng vào vị trí của Sommer nên không gây nguy hiểm.Bất ngờ xảy ra ở phút 27. Rodri nhận bóng ở trung lộ sau đường chuyền từ bên phải vào, anh ngoặt bóng loại bỏ cầu thủ của đội khách rồi thực hiện cú cứa lòng chân trái từ cự ly chừng 21m, bóng đi xoáy vào góc cao bên phải cầu môn khiến Sommer không thể cản phá.  Sau bàn thắng, Man City vẫn chơi nhỉnh hơn Bayern song họ không có thêm bàn thắng trong hiệp một. Đoàn quân của Thomas Tuchel tấn công không sắc nét trong hiệp đấu đầu tiên, họ chỉ có 4 pha dứt điểm và đều không trúng cầu môn đội chủ nhà. Bước vào hiệp hai, Bayern Munich đẩy cao đội hình tấn công, gia tăng kiểm soát bóng để tìm kiếm bàn thắng gỡ hòa. Phút 46, Sane tung cú sút như búa bổ từ bên trái trung lộ, bóng đi vào góc trái, Ederson đã chủ động khép góc nên cản phá thành công. Khi hàng công của đội khách còn chưa tìm được vào khung thành Man City, hàng thủ đã mắc sai lầm. Phút 70, Grealish cướp bóng trong chân của Upamecano rồi đánh gót cho Haaland băng xuống bên trái. Cầu thủ người Na Uy tạt bóng chính xác để Bernardo Silva băng vào đánh đầu tung lưới Bayern nâng tỷ số lên 2-0.Có hai bàn thắng trong tay, Man City chơi rất thoải mái. Phút 76, Alvarez tạt bóng mạnh từ bên phải vào trong vòng cấm địa, Stones vẫn đứng trong vòng cấm địa đối phương sau khi tham gia pha tấn công trước đó, anh đã đánh đầu chuyền bóng sang phải để Haaland ập vào đệm bóng từ cự ly gần nâng tỷ số lên 3-0. Man City vẫn khát khao ghi bàn trong những phút còn lại, tiếc rằng họ không thành công. Tuy nhiên, với khoảng cách ba bàn sau lượt đi, Man City sẽ thoải mái bước vào trận lượt về vào tuần sau."]
lines = ViTokenizer.tokenize(text[0])
text[0] = lines

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h3>1, split sentence</h3>

In [39]:
sentences = nltk.sent_tokenize(text[0])

# In ra các câu được tách
for sentence in sentences:
    print(sentence)

Man City đang có phong_độ tốt trước trận tứ_kết lượt đi Champions_League trên sân_nhà gặp Bayern .
Đoàn_quân của Pep_Guardiola hiểu rằng họ cần có một kết_quả tốt trước khi làm_khách trên sân đối_thủ ở lượt về , vì_vậy Man City chủ_động dâng cao đội_hình tấn_công gây sức_ép lên đối_phương ngay sau tiếng còi khai_cuộc .
Phút 5 , De Bruyne chuyền bóng cho Haaland từ bên trái .
Cầu_thủ người Na Uy bứt tốc nhanh và dứt_điểm uy_lực từ bên trái vòng cấm_địa , bóng đi vọt xà .
9 phút sau , Haaland khiến cổ_động_viên Bayern thót_tim khi lao vào rất nhanh tranh bóng với thủ_thành Sommer ngay trên vạch vôi .
Thủ_môn đội khách xử_lý chậm , anh phá bóng đập chân Haaland nhưng may_mắn trái bóng đã văng ra biên .
Haaland thi_đấu nổi_bật trên hàng công của Man City , các đồng_đội cũng tập_trung nhiều bóng cho anh .
Phút 22 , Haaland thoải_mái dứt_điểm ở đầu vòng cấm_địa .
Tiếc rằng cú sút của anh hơi nhẹ , bóng đi chìm và hướng thẳng vào vị_trí của Sommer nên không gây nguy_hiểm .
Bất_ngờ xảy ra ở ph

<h3>2, change sentence to vector of word</h3>

In [40]:
sentences_count_type = count_vect.transform(sentences)
print(type(sentences_count_type))

<class 'scipy.sparse._csr.csr_matrix'>


<h3>3, Build Kmeans model </h3>

In [41]:
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(sentences_count_type)

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [42]:
pickle.dump(kmeans, open("kmeans_object.pkl", "wb"))

<h3>4, summary</h3>

<p>điểm lấy ra để đại diện cho các câu là điểm gần centroid nhất</p>

In [43]:
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np 

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    #dùng để sắp xếp thứ tự các câu, = trung bình thứ tự xuất hiện
    avg.append(np.mean(idx))
#chọn câu đại diện là câu gần trung tâm nhất
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_count_type)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [44]:
print(summary)

Bất_ngờ xảy ra ở phút 27 . Rodri nhận bóng ở trung_lộ sau đường chuyền từ bên phải vào , anh ngoặt bóng loại_bỏ cầu_thủ của đội khách rồi thực_hiện cú cứa lòng chân trái từ cự_ly chừng 21m , bóng đi xoáy vào góc cao bên phải cầu_môn khiến Sommer không_thể cản_phá . Haaland thi_đấu nổi_bật trên hàng công của Man City , các đồng_đội cũng tập_trung nhiều bóng cho anh . Đoàn_quân của Thomas Tuchel tấn_công không sắc nét trong hiệp_đấu đầu_tiên , họ chỉ có 4 pha dứt_điểm và đều không trúng cầu_môn đội chủ nhà . Phút 76 , Alvarez tạt bóng mạnh từ bên phải vào trong vòng cấm_địa , Stones vẫn đứng trong vòng cấm_địa đối_phương sau khi tham_gia pha tấn_công trước đó , anh đã đánh_đầu chuyền bóng sang phải để Haaland ập vào đệm_bóng từ cự_ly gần nâng tỷ_số lên 3 - 0 .
